In [12]:
from fastmcp import Client
from rich.pretty import pprint
from typing import Dict

In [5]:
client = Client("http://localhost:8001/mcp")

async with client:

    tools = await client.list_tools()

In [11]:
pprint(tools)
pprint(tools[0].description)

[
│   Tool(
│   │   name='get_formatted_item_context',
│   │   title=None,
│   │   description='Get the top k context, each representing an inventory item for a given query.\n\nArgs:\n    query: The query to get the top k context for\n    top_k: The number of context chunks to retrieve, works best with 5 or more\n\nReturns:\n    A string of the top k context chunks with IDs prepending each chunk, each representing an inventory item for a given query.',
│   │   inputSchema={
│   │   │   'properties': {
│   │   │   │   'query': {'title': 'Query', 'type': 'string'},
│   │   │   │   'top_k': {'default': 5, 'title': 'Top K', 'type': 'integer'}
│   │   │   },
│   │   │   'required': ['query'],
│   │   │   'type': 'object'
│   │   },
│   │   outputSchema={
│   │   │   'properties': {'result': {'title': 'Result', 'type': 'string'}},
│   │   │   'required': ['result'],
│   │   │   'title': '_WrappedResult',
│   │   │   'type': 'object',
│   │   │   'x-fastmcp-wrap-result': True
│   │   },
│   │   annotations=None,
│   │   meta={'_fastmcp': {'tags': []}}
│   )
]

'Get the top k context, each representing an inventory item for a given query.\n\nArgs:\n    query: The query to get the top k context for\n    top_k: The number of context chunks to retrieve, works best with 5 or more\n\nReturns:\n    A string of the top k context chunks with IDs prepending each chunk, each representing an inventory item for a given query.'

In [13]:
def parse_docstring_params(docstring: str) -> Dict[str, str]:
    """Extract parameter descriptions from docstring (handles both Args: and Parameters: formats)."""
    params = {}
    lines = docstring.split('\n')
    in_params = False
    current_param = None
    
    for line in lines:
        stripped = line.strip()
        
        # Check for parameter section start
        if stripped in ['Args:', 'Arguments:', 'Parameters:', 'Params:']:
            in_params = True
            current_param = None
        elif stripped.startswith('Returns:') or stripped.startswith('Raises:'):
            in_params = False
        elif in_params:
            # Parse parameter line (handles "param: desc" and "- param: desc" formats)
            if ':' in stripped and (stripped[0].isalpha() or stripped.startswith(('-', '*'))):
                param_name = stripped.lstrip('- *').split(':')[0].strip()
                param_desc = ':'.join(stripped.lstrip('- *').split(':')[1:]).strip()
                params[param_name] = param_desc
                current_param = param_name
            elif current_param and stripped:
                # Continuation of previous parameter description
                params[current_param] += ' ' + stripped
    
    return params

In [14]:
parse_docstring_params(tools[0].description)

{'query': 'The query to get the top k context for', 'top_k': 'The number of context chunks to retrieve, works best with 5 or more'}

In [17]:
result = {
    'name': tools[0].name,
    'description': tools[0].description.split("\n\n")[0],
    'required': tools[0].inputSchema.get('required', []),
    'returns': {
        'type': 'string',
        'description': tools[0].description.split("Returns:")[1].strip(),
    },
    'parameters': {
        'type': 'object',
        'properties': {}
    }
}

property_descriptions = parse_docstring_params(tools[0].description)
properties = tools[0].inputSchema.get('properties', {})

for key, value in properties.items():
    properties[key]['description'] = property_descriptions.get(key, '')

result['parameters']['properties'] = properties


In [18]:
pprint(result)

{
│   'name': 'get_formatted_item_context',
│   'description': 'Get the top k context, each representing an inventory item for a given query.',
│   'required': ['query'],
│   'returns': {
│   │   'type': 'string',
│   │   'description': 'A string of the top k context chunks with IDs prepending each chunk, each representing an inventory item for a given query.'
│   },
│   'parameters': {
│   │   'type': 'object',
│   │   'properties': {
│   │   │   'query': {
│   │   │   │   'title': 'Query',
│   │   │   │   'type': 'string',
│   │   │   │   'description': 'The query to get the top k context for'
│   │   │   },
│   │   │   'top_k': {
│   │   │   │   'default': 5,
│   │   │   │   'title': 'Top K',
│   │   │   │   'type': 'integer',
│   │   │   │   'description': 'The number of context chunks to retrieve, works best with 5 or more'
│   │   │   }
│   │   }
│   }
}

In [19]:
client = Client("http://localhost:8002/mcp")

async with client:

    tools = await client.list_tools()

pprint(tools)

[
│   Tool(
│   │   name='get_formatted_review_context',
│   │   title=None,
│   │   description='Get the top k reviews matching a query for a list of prefiltered items.\n\nArgs:\n    query: The query to get the top k reviews for\n    item_list: The list of item IDs to prefilter before running the query\n    top_k: The number of reviews to retrieve, this should be at least 20 if multiple items are prefiltered\n\nReturns:\n    A string of the top k context chunks with IDs prepending each chunk, each representing an inventory item for a given query.',
│   │   inputSchema={
│   │   │   'properties': {
│   │   │   │   'query': {'title': 'Query', 'type': 'string'},
│   │   │   │   'item_list': {'items': {'type': 'string'}, 'title': 'Item List', 'type': 'array'},
│   │   │   │   'top_k': {'default': 20, 'title': 'Top K', 'type': 'integer'}
│   │   │   },
│   │   │   'required': ['query', 'item_list'],
│   │   │   'type': 'object'
│   │   },
│   │   outputSchema={
│   │   │   'properties': {'result': {'title': 'Result', 'type': 'string'}},
│   │   │   'required': ['result'],
│   │   │   'title': '_WrappedResult',
│   │   │   'type': 'object',
│   │   │   'x-fastmcp-wrap-result': True
│   │   },
│   │   annotations=None,
│   │   meta={'_fastmcp': {'tags': []}}
│   )
]

In [20]:
async def get_tool_descriptions_from_mcp_servers(mcp_servers: list[str]) -> list[dict]:

    tool_descriptions = []

    for mcp_server in mcp_servers:
        client = Client(mcp_server)
        async with client:
            tools = await client.list_tools()
            for tool in tools:
                tool_description = {
                    'name': tool.name,
                    'description': tool.description.split("\n\n")[0],
                    'required': tool.inputSchema.get('required', []),
                    'returns': {
                        'type': 'string',
                        'description': tool.description.split("Returns:")[1].strip(),
                    },
                    'parameters': {
                        'type': 'object',
                        'properties': {}
                    }
                }
                property_descriptions = parse_docstring_params(tool.description)
                properties = tool.inputSchema.get('properties', {})

                for key, value in properties.items():
                    properties[key]['description'] = property_descriptions.get(key, '')

                tool_description['parameters']['properties'] = properties
                tool_descriptions.append(tool_description)

    return tool_descriptions

In [ ]:
mcp_servers = [
    "http://localhost:8001/mcp",
    "http://localhost:8002/mcp"
]


tool_descriptions = await get_tool_descriptions_from_mcp_servers(mcp_servers)

pprint(tool_descriptions)

<coroutine object get_tool_descriptions_from_mcp_servers at 0x1164baf80>